# Load medical images

This notebook introduces how to easily load different formats of medical images in MONAI and execute many additional operations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/modules/load_medical_images.ipynb)

## Setup environment

In [1]:
%pip install -q "monai[itk, nibabel, pillow]"

Note: you may need to restart the kernel to use updated packages.


## Setup imports

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import numpy as np
import itk
from PIL import Image
import tempfile
from monai.data import ITKReader, NibabelReader, PILReader
from monai.transforms import LoadImage, LoadImaged, AddChanneld, Resized, ToTensord, Compose
from monai.config import print_config

print_config()

## Load Nifti image with default image reader
MONAI leverages `ITK` as the default image reader, it can support most of the common medical image formats.
More details, please check: https://github.com/InsightSoftwareConsortium/ITK/tree/master/Modules/IO

In [2]:
# generate 3D test images
tempdir = tempfile.mkdtemp()
test_image = np.random.rand(64, 128, 96)
filename = os.path.join(tempdir, "test_image.nii.gz")
itk_np_view = itk.image_view_from_array(test_image)
itk.imwrite(itk_np_view, filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(64, 128, 96)
meta data:{'ITK_FileNotes': '', 'aux_file': '', 'bitpix': '64', 'cal_max': '0', 'cal_min': '0', 'datatype': '64', 'descrip': '', 'dim[0]': '3', 'dim[1]': '96', 'dim[2]': '128', 'dim[3]': '64', 'dim[4]': '1', 'dim[5]': '1', 'dim[6]': '1', 'dim[7]': '1', 'dim_info': '0', 'intent_code': '0', 'intent_name': '', 'intent_p1': '0', 'intent_p2': '0', 'intent_p3': '0', 'nifti_type': '1', 'pixdim[0]': '0', 'pixdim[1]': '1', 'pixdim[2]': '1', 'pixdim[3]': '1', 'pixdim[4]': '0', 'pixdim[5]': '0', 'pixdim[6]': '0', 'pixdim[7]': '0', 'qform_code': '1', 'qform_code_name': 'NIFTI_XFORM_SCANNER_ANAT', 'qoffset_x': '-0', 'qoffset_y': '-0', 'qoffset_z': '0', 'quatern_b': '0', 'quatern_c': '0', 'quatern_d': '1', 'scl_inter': '0', 'scl_slope': '1', 'sform_code': '0', 'sform_code_name': 'NIFTI_XFORM_UNKNOWN', 'slice_code': '0', 'slice_duration': '0', 'slice_end': '0', 'slice_start': '0', 'srow_x': '0 0 0 0', 'srow_y': '0 0 0 0', 'srow_z': '0 0 0 0', 'toffset': '0', 'vox_offset

## Load a list of Nifti images and stack as 1 training item
Loading a list of files, stack them together and add a new dimension as first dimension.

And use the meta data of the first image to represent the stacked result.

In [3]:
filenames = ["test_image.nii.gz", "test_image2.nii.gz", "test_image3.nii.gz"]
for i, name in enumerate(filenames):
    filenames[i] = os.path.join(tempdir, name)
    itk_np_view = itk.image_view_from_array(test_image)
    itk.imwrite(itk_np_view, filenames[i])
data, meta = LoadImage()(filenames)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(3, 64, 128, 96)
meta data:{'ITK_FileNotes': '', 'aux_file': '', 'bitpix': '64', 'cal_max': '0', 'cal_min': '0', 'datatype': '64', 'descrip': '', 'dim[0]': '3', 'dim[1]': '96', 'dim[2]': '128', 'dim[3]': '64', 'dim[4]': '1', 'dim[5]': '1', 'dim[6]': '1', 'dim[7]': '1', 'dim_info': '0', 'intent_code': '0', 'intent_name': '', 'intent_p1': '0', 'intent_p2': '0', 'intent_p3': '0', 'nifti_type': '1', 'pixdim[0]': '0', 'pixdim[1]': '1', 'pixdim[2]': '1', 'pixdim[3]': '1', 'pixdim[4]': '0', 'pixdim[5]': '0', 'pixdim[6]': '0', 'pixdim[7]': '0', 'qform_code': '1', 'qform_code_name': 'NIFTI_XFORM_SCANNER_ANAT', 'qoffset_x': '-0', 'qoffset_y': '-0', 'qoffset_z': '0', 'quatern_b': '0', 'quatern_c': '0', 'quatern_d': '1', 'scl_inter': '0', 'scl_slope': '1', 'sform_code': '0', 'sform_code_name': 'NIFTI_XFORM_UNKNOWN', 'slice_code': '0', 'slice_duration': '0', 'slice_end': '0', 'slice_start': '0', 'srow_x': '0 0 0 0', 'srow_y': '0 0 0 0', 'srow_z': '0 0 0 0', 'toffset': '0', 'vox_off

## Load 3D image in DICOM format

In [4]:
filename = os.path.join(tempdir, "test_image.dcm")
dcm_image = np.random.randint(256, size=(64, 128, 96)).astype(np.uint8())
itk_np_view = itk.image_view_from_array(dcm_image)
itk.imwrite(itk_np_view, filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(64, 128, 96)
meta data:{'0008|0016': '1.2.840.10008.5.1.4.1.1.7.2', '0008|0018': '1.2.826.0.1.3680043.2.1125.1.64732409224407717340688313759116819', '0008|0020': '20200917', '0008|0030': '012951.461675 ', '0008|0050': '', '0008|0060': 'OT', '0008|0090': '', '0010|0010': '', '0010|0020': '', '0010|0030': '', '0010|0040': '', '0020|000d': '1.2.826.0.1.3680043.2.1125.1.22199909824554379284164927198357157', '0020|000e': '1.2.826.0.1.3680043.2.1125.1.64330868579047092348166172965815757', '0020|0010': '', '0020|0011': '', '0020|0013': '', '0020|0052': '1.2.826.0.1.3680043.2.1125.1.36887462277578362563842268362959985', '0028|0002': '1', '0028|0004': 'MONOCHROME2 ', '0028|0008': '64', '0028|0009': '(5200,9230)', '0028|0010': '128', '0028|0011': '96', '0028|0100': '8', '0028|0101': '8', '0028|0102': '7', '0028|0103': '0', '0028|1052': '0 ', '0028|1053': '1 ', '0028|1054': 'US', 'origin': array([0., 0., 0.]), 'spacing': array([1., 1., 1.]), 'direction': array([[1., 0., 0.],
   

## Load a list of DICOM images and stack as 1 training item
Loading a list of files, stack them together and add a new dimension as first dimension.

And use the meta data of the first image to represent the stacked result.

In [5]:
filenames = ["test_image.dcm", "test_image2.dcm", "test_image3.dcm"]
for i, name in enumerate(filenames):
    filenames[i] = os.path.join(tempdir, name)
    itk_np_view = itk.image_view_from_array(dcm_image)
    itk.imwrite(itk_np_view, filenames[i])
data, meta = LoadImage()(filenames)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(3, 64, 128, 96)
meta data:{'0008|0016': '1.2.840.10008.5.1.4.1.1.7.2', '0008|0018': '1.2.826.0.1.3680043.2.1125.1.55124182629917658983046437541692429', '0008|0020': '20200917', '0008|0030': '012953.377970 ', '0008|0050': '', '0008|0060': 'OT', '0008|0090': '', '0010|0010': '', '0010|0020': '', '0010|0030': '', '0010|0040': '', '0020|000d': '1.2.826.0.1.3680043.2.1125.1.54570928137383298712421968711546946', '0020|000e': '1.2.826.0.1.3680043.2.1125.1.79105954593572939973262028207196688', '0020|0010': '', '0020|0011': '', '0020|0013': '', '0020|0052': '1.2.826.0.1.3680043.2.1125.1.37569814707528773722727172695078933', '0028|0002': '1', '0028|0004': 'MONOCHROME2 ', '0028|0008': '64', '0028|0009': '(5200,9230)', '0028|0010': '128', '0028|0011': '96', '0028|0100': '8', '0028|0101': '8', '0028|0102': '7', '0028|0103': '0', '0028|1052': '0 ', '0028|1053': '1 ', '0028|1054': 'US', 'origin': array([0., 0., 0.]), 'spacing': array([1., 1., 1.]), 'direction': array([[1., 0., 0.],


## Load 2D image in PNG format

In [6]:
test_image = np.random.randint(0, 256, size=[128, 256])
filename = os.path.join(tempdir, "test_image.png")
Image.fromarray(test_image.astype("uint8")).save(filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(128, 256)
meta data:{'origin': array([0., 0.]), 'spacing': array([1., 1.]), 'direction': array([[1., 0.],
       [0., 1.]]), 'original_affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'spatial_shape': [128, 256], 'filename_or_obj': '/tmp/tmpq5gymhdr/test_image.png'}


## Load image with specified image reader
The `LoadImage` transforms can automatically choose readers based on the supported subffixes and in below order:
- User specified reader at runtime when call this loader.
- Registered readers from the first to the last in list, user can register reader when initializing or at runtime.
- Default ITK reader.

And we can set additional parameters for the image readers, for example, set `c_order_axis_indexing=True` for `ITKReader`, this parameter will pass to ITK `read()` function later.

In [7]:
loader = LoadImage()
loader.register(ITKReader())
data, meta = loader(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(128, 256)
meta data:{'origin': array([0., 0.]), 'spacing': array([1., 1.]), 'direction': array([[1., 0.],
       [0., 1.]]), 'original_affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'spatial_shape': [128, 256], 'filename_or_obj': '/tmp/tmpq5gymhdr/test_image.png'}


## Load image and execute additional operations
Some image readers can support additional operations after reading the image from file.

For example, we can set a converter for PILReader: `PILReader(converter=lambda image: image.convert("LA"))`.

In [8]:
loader = LoadImage(PILReader(converter=lambda image: image.convert("LA")))
data, meta = loader(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(128, 256, 2)
meta data:{'format': None, 'mode': 'LA', 'width': 256, 'height': 128, 'info': {}, 'spatial_shape': [256, 128], 'filename_or_obj': '/tmp/tmpq5gymhdr/test_image.png'}


## Connect `LoadImage` with other transforms
It's very easy to connect `LoadImage` transform with other transforms to construct a transform chain.

In [9]:
transform = Compose([
    LoadImaged(keys="image"),
    AddChanneld(keys="image"),
    Resized(keys="image", spatial_size=[64, 64]),
    ToTensord("image"),
])
test_data = {"image": filename}
result = transform(test_data)
print(f"image data shape:{result['image'].shape}")
print(f"meta data:{result['image_meta_dict']}")

image data shape:torch.Size([1, 64, 64])
meta data:{'origin': array([0., 0.]), 'spacing': array([1., 1.]), 'direction': array([[1., 0.],
       [0., 1.]]), 'original_affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'spatial_shape': [128, 256], 'filename_or_obj': '/tmp/tmpq5gymhdr/test_image.png'}


## Cleanup data directory

Remove directory if a temporary was used.

In [10]:
shutil.rmtree(tempdir)